In [1]:
import torch
import torch.nn as nn


class Backbone(nn.Module):
    """Base class for backbone networks. Handles freezing layers etc.
    args:
        frozen_layers  -  Name of layers to freeze. Either list of strings, 'none' or 'all'. Default: 'none'.
    """
    def __init__(self, frozen_layers=()):
        super().__init__()

        if isinstance(frozen_layers, str):
            if frozen_layers.lower() == 'none':
                frozen_layers = ()
            elif frozen_layers.lower() != 'all':
                raise ValueError('Unknown option for frozen layers: \"{}\". Should be \"all\", \"none\" or list of layer names.'.format(frozen_layers))

        self.frozen_layers = frozen_layers
        self._is_frozen_nograd = False


    def train(self, mode=True):
        super().train(mode)
        if mode == True:
            self._set_frozen_to_eval()
        if not self._is_frozen_nograd:
            self._set_frozen_to_nograd()
            self._is_frozen_nograd = True
        return self


    def _set_frozen_to_eval(self):
        if isinstance(self.frozen_layers, str) and self.frozen_layers.lower() == 'all':
            self.eval()
        else:
            for layer in self.frozen_layers:
                getattr(self, layer).eval()


    def _set_frozen_to_nograd(self):
        if isinstance(self.frozen_layers, str) and self.frozen_layers.lower() == 'all':
            for p in self.parameters():
                p.requires_grad_(False)
        else:
            for layer in self.frozen_layers:
                for p in getattr(self, layer).parameters():
                    p.requires_grad_(False)
import numpy as np
import torch
import torch.nn as nn


class channel_selection(nn.Module):
    """
    Select channels from the output of BatchNorm2d layer. It should be put directly after BatchNorm2d layer.
    The output shape of this layer is determined by the number of 1 in `self.indexes`.
    """
    def __init__(self, num_channels):
        """
        Initialize the `indexes` with all one vector with the length same as the number of channels.
        During pruning, the places in `indexes` which correpond to the channels to be pruned will be set to 0.
        """
        super(channel_selection, self).__init__()
        self.indexes = nn.Parameter(torch.ones(num_channels))

    def forward(self, input_tensor):
        """
        Parameter
        ---------
        input_tensor: (N,C,H,W). It should be the output of BatchNorm2d layer.
        """
        selected_index = np.squeeze(np.argwhere(self.indexes.data.cpu().numpy()))
        if selected_index.size == 1:
            selected_index = np.resize(selected_index, (1,)) 
        output = input_tensor[:, :, :, :]
        return output


In [2]:
import math
import torch.nn as nn
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torchvision.models.resnet import model_urls
# from .base import Backbone
# from .channel_selection import channel_selection



def conv3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, bias=False, dilation=dilation)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1, use_bn=True):
        super(BasicBlock, self).__init__()
        self.use_bn = use_bn
        self.conv1 = conv3x3(inplanes, planes, stride, dilation=dilation)

        if use_bn:
            self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, dilation=dilation)

        if use_bn:
            self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        if self.use_bn:
            out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)

        if self.use_bn:
            out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=dilation, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.select = channel_selection(planes*4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
#         out = self.select(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(Backbone):
    """ ResNet network module. Allows extracting specific feature blocks."""
    def __init__(self, block, layers, output_layers, num_classes=1000, inplanes=64, dilation_factor=1, frozen_layers=()):
        self.inplanes = inplanes
        super(ResNet, self).__init__(frozen_layers=frozen_layers)
        self.output_layers = output_layers
        self.conv1 = nn.Conv2d(3, inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        stride = [1 + (dilation_factor < l) for l in (8, 4, 2)]
        self.layer1 = self._make_layer(block, inplanes, layers[0], dilation=max(dilation_factor//8, 1))
        self.layer2 = self._make_layer(block, inplanes*2, layers[1], stride=stride[0], dilation=max(dilation_factor//4, 1))
        self.layer3 = self._make_layer(block, inplanes*4, layers[2], stride=stride[1], dilation=max(dilation_factor//2, 1))
        self.layer4 = self._make_layer(block, inplanes*8, layers[3], stride=stride[2], dilation=dilation_factor)

        out_feature_strides = {'conv1': 4, 'layer1': 4, 'layer2': 4*stride[0], 'layer3': 4*stride[0]*stride[1],
                               'layer4': 4*stride[0]*stride[1]*stride[2]}

        # TODO better way?
        if isinstance(self.layer1[0], BasicBlock):
            out_feature_channels = {'conv1': inplanes, 'layer1': inplanes, 'layer2': inplanes*2, 'layer3': inplanes*4,
                               'layer4': inplanes*8}
        elif isinstance(self.layer1[0], Bottleneck):
            base_num_channels = 4 * inplanes
            out_feature_channels = {'conv1': inplanes, 'layer1': base_num_channels, 'layer2': base_num_channels * 2,
                                    'layer3': base_num_channels * 4, 'layer4': base_num_channels * 8}
        else:
            raise Exception('block not supported')

        self._out_feature_strides = out_feature_strides
        self._out_feature_channels = out_feature_channels

        # self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(inplanes*8 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def out_feature_strides(self, layer=None):
        if layer is None:
            return self._out_feature_strides
        else:
            return self._out_feature_strides[layer]

    def out_feature_channels(self, layer=None):
        if layer is None:
            return self._out_feature_channels
        else:
            return self._out_feature_channels[layer]

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
                channel_selection(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, dilation=dilation))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _add_output_and_check(self, name, x, outputs, output_layers):
        if name in output_layers:
            outputs[name] = x
        return len(output_layers) == len(outputs)

    def forward(self, x, output_layers=None):
        """ Forward pass with input x. The output_layers specify the feature blocks which must be returned """
        outputs = OrderedDict()

        if output_layers is None:
            output_layers = self.output_layers

        x = self.conv1(x)
        x = self.bn1(x) ####### select daal dena
        x = self.relu(x)

        if self._add_output_and_check('conv1', x, outputs, output_layers):
            return outputs

        x = self.maxpool(x)

        x = self.layer1(x)

        if self._add_output_and_check('layer1', x, outputs, output_layers):
            return outputs

        x = self.layer2(x)

        if self._add_output_and_check('layer2', x, outputs, output_layers):
            return outputs

        x = self.layer3(x)

        if self._add_output_and_check('layer3', x, outputs, output_layers):
            return outputs

        x = self.layer4(x)

        if self._add_output_and_check('layer4', x, outputs, output_layers):
            return outputs

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        if self._add_output_and_check('fc', x, outputs, output_layers):
            return outputs

        if len(output_layers) == 1 and output_layers[0] == 'default':
            return x

        raise ValueError('output_layer is wrong.')


def resnet_baby(output_layers=None, pretrained=False, inplanes=16, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, inplanes=inplanes, **kwargs)

    if pretrained:
        raise NotImplementedError
    return model


def resnet18(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, **kwargs)

    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet50(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(Bottleneck, [3, 4, 6, 3], output_layers, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50'], progress = False), strict = False )
    return model


In [3]:
model = resnet50()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_selection()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 

In [4]:
cd /workspace/pytracking


/workspace/pytracking


In [5]:
from ltr.admin.loading import torch_load_legacy

In [6]:
ckpt = torch_load_legacy('../tracking_datasets/saved_ckpts/ltr/dimp/sparse-pretrained/dimp50/DiMPnet_ep0050.pth.tar')['net']

from collections import OrderedDict
new_state = OrderedDict()

for key, value in ckpt.items():
    key = key[18:] # remove `module.`
    new_state[key] = value
model.load_state_dict(new_state, strict = False )

_IncompatibleKeys(missing_keys=['layer1.0.select.indexes', 'layer1.0.downsample.2.indexes', 'layer1.1.select.indexes', 'layer1.2.select.indexes', 'layer2.0.select.indexes', 'layer2.0.downsample.2.indexes', 'layer2.1.select.indexes', 'layer2.2.select.indexes', 'layer2.3.select.indexes', 'layer3.0.select.indexes', 'layer3.0.downsample.2.indexes', 'layer3.1.select.indexes', 'layer3.2.select.indexes', 'layer3.3.select.indexes', 'layer3.4.select.indexes', 'layer3.5.select.indexes', 'layer4.0.select.indexes', 'layer4.0.downsample.2.indexes', 'layer4.1.select.indexes', 'layer4.2.select.indexes'], unexpected_keys=['initializer.filter_conv.weight', 'initializer.filter_conv.bias', 'optimizer.log_step_length', 'optimizer.filter_reg', 'optimizer.label_map_predictor.weight', 'optimizer.target_mask_predictor.0.weight', 'optimizer.spatial_weight_predictor.weight', '_extractor.0.weight', '_1r.0.weight', '_1r.0.bias', '_1r.1.weight', '_1r.1.bias', '_1r.1.running_mean', '_1r.1.running_var', '_1r.1.num_b

In [7]:
for idx, m in model.named_modules():
    if isinstance(m, nn.BatchNorm2d):
        
        if idx.split('.')[0] == 'layer4':
            print(idx, '****')
            print(m.weight.shape)
            torch.nn.init.ones_(m.weight)
            print(m.weight.shape)
        print(idx)

bn1
layer1.0.bn1
layer1.0.bn2
layer1.0.bn3
layer1.0.downsample.1
layer1.1.bn1
layer1.1.bn2
layer1.1.bn3
layer1.2.bn1
layer1.2.bn2
layer1.2.bn3
layer2.0.bn1
layer2.0.bn2
layer2.0.bn3
layer2.0.downsample.1
layer2.1.bn1
layer2.1.bn2
layer2.1.bn3
layer2.2.bn1
layer2.2.bn2
layer2.2.bn3
layer2.3.bn1
layer2.3.bn2
layer2.3.bn3
layer3.0.bn1
layer3.0.bn2
layer3.0.bn3
layer3.0.downsample.1
layer3.1.bn1
layer3.1.bn2
layer3.1.bn3
layer3.2.bn1
layer3.2.bn2
layer3.2.bn3
layer3.3.bn1
layer3.3.bn2
layer3.3.bn3
layer3.4.bn1
layer3.4.bn2
layer3.4.bn3
layer3.5.bn1
layer3.5.bn2
layer3.5.bn3
layer4.0.bn1 ****
torch.Size([512])
torch.Size([512])
layer4.0.bn1
layer4.0.bn2 ****
torch.Size([512])
torch.Size([512])
layer4.0.bn2
layer4.0.bn3 ****
torch.Size([2048])
torch.Size([2048])
layer4.0.bn3
layer4.0.downsample.1 ****
torch.Size([2048])
torch.Size([2048])
layer4.0.downsample.1
layer4.1.bn1 ****
torch.Size([512])
torch.Size([512])
layer4.1.bn1
layer4.1.bn2 ****
torch.Size([512])
torch.Size([512])
layer4.1.bn2

In [8]:
total = 0

for idx, m in model.named_modules():
    if isinstance(m, nn.BatchNorm2d):
        if idx.split('.')[0] != 'layer4':
            
            total += m.weight.data.shape[0]

bn = torch.zeros(total)
index = 0
for (idx, m) in model.named_modules():
    if isinstance(m, nn.BatchNorm2d):
        if idx.split('.')[0] != 'layer4':
            size = m.weight.data.shape[0]
            bn[index:(index+size)] = m.weight.data.abs().clone()
            index += size

y, i = torch.sort(bn)
thre_index = int(total * 0.5)
thre = y[thre_index]


pruned = 0
cfg = []
cfg_mask = []
modules = list(model.modules())
for k, (idx, m) in enumerate(model.named_modules()):
    if isinstance(m, nn.BatchNorm2d) :
        weight_copy = m.weight.data.abs().clone()
        mask = weight_copy.gt(thre).float()
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        m.weight.data.mul_(mask)
        m.bias.data.mul_(mask)
        if isinstance(modules[k-3],channel_selection)==False:
            cfg.append(int(torch.sum(mask)))
        cfg_mask.append(mask.clone())
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))
    elif isinstance(m, nn.MaxPool2d):
        cfg.append('M')

pruned_ratio = pruned/total


layer index: 2 	 total channel: 64 	 remaining channel: 62
layer index: 8 	 total channel: 64 	 remaining channel: 47
layer index: 10 	 total channel: 64 	 remaining channel: 53
layer index: 12 	 total channel: 256 	 remaining channel: 133
layer index: 17 	 total channel: 256 	 remaining channel: 200
layer index: 21 	 total channel: 64 	 remaining channel: 56
layer index: 23 	 total channel: 64 	 remaining channel: 60
layer index: 25 	 total channel: 256 	 remaining channel: 63
layer index: 30 	 total channel: 64 	 remaining channel: 63
layer index: 32 	 total channel: 64 	 remaining channel: 63
layer index: 34 	 total channel: 256 	 remaining channel: 67
layer index: 40 	 total channel: 128 	 remaining channel: 127
layer index: 42 	 total channel: 128 	 remaining channel: 128
layer index: 44 	 total channel: 512 	 remaining channel: 221
layer index: 49 	 total channel: 512 	 remaining channel: 224
layer index: 53 	 total channel: 128 	 remaining channel: 43
layer index: 55 	 total cha

In [26]:
(cfg)

[62,
 47,
 53,
 133,
 200,
 60,
 63,
 63,
 63,
 67,
 127,
 128,
 221,
 224,
 109,
 144,
 116,
 118,
 168,
 123,
 127,
 147,
 256,
 255,
 569,
 299,
 247,
 247,
 216,
 247,
 181,
 233,
 242,
 208,
 238,
 242,
 254,
 251,
 254,
 329,
 512,
 512,
 2048,
 2048,
 512,
 2048,
 512,
 512,
 2048]

In [10]:
y

tensor([4.5469e-12, 1.3583e-10, 2.2824e-10,  ..., 5.0624e-01, 5.0981e-01,
        5.2134e-01])

In [11]:
cfg

[62,
 'M',
 47,
 53,
 133,
 200,
 60,
 63,
 63,
 63,
 67,
 127,
 128,
 221,
 224,
 109,
 144,
 116,
 118,
 168,
 123,
 127,
 147,
 256,
 255,
 569,
 299,
 247,
 247,
 216,
 247,
 181,
 233,
 242,
 208,
 238,
 242,
 254,
 251,
 254,
 329,
 512,
 512,
 2048,
 2048,
 512,
 2048,
 512,
 512,
 2048]

In [12]:
len(cfg)

50

In [13]:
# model

In [14]:
import math
import torch.nn as nn
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torchvision.models.resnet import model_urls
# from .base import Backbone
# from .channel_selection import channel_selection



def conv3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, bias=False, dilation=dilation)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1, use_bn=True):
        super(BasicBlock, self).__init__()
        self.use_bn = use_bn
        self.conv1 = conv3x3(inplanes, planes, stride, dilation=dilation)

        if use_bn:
            self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, dilation=dilation)

        if use_bn:
            self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        if self.use_bn:
            out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)

        if self.use_bn:
            out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cfg, stride=1, downsample=None, dilation=1,):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(cfg[0],cfg[1], kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(cfg[1])
        self.conv2 = nn.Conv2d(cfg[1], cfg[2], kernel_size=3, stride=stride,
                               padding=dilation, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(cfg[2])
        self.conv3 = nn.Conv2d(cfg[2], planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.select = channel_selection(planes*4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
#         out = self.select(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(Backbone):
    """ ResNet network module. Allows extracting specific feature blocks."""
    def __init__(self, block, layers, output_layers,cfg = None , num_classes=1000, inplanes=64, dilation_factor=1, frozen_layers=()):
        self.inplanes = inplanes
        super(ResNet, self).__init__(frozen_layers=frozen_layers)
        self.output_layers = output_layers
        self.conv1 = nn.Conv2d(3, cfg[0], kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(cfg[0])

        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        stride = [1 + (dilation_factor < l) for l in (8, 4, 2)]
        self.layer1 = self._make_layer(block, inplanes, layers[0], dilation=max(dilation_factor//8, 1), cfg = cfg[0:3*layers[0]])
        self.layer2 = self._make_layer(block, inplanes*2, layers[1], stride=stride[0], dilation=max(dilation_factor//4, 1), cfg = cfg[3*layers[0]:3*layers[1]+3*layers[0]])
        self.layer3 = self._make_layer(block, inplanes*4, layers[2], stride=stride[1], dilation=max(dilation_factor//2, 1), cfg = cfg[3*layers[1]+3*layers[0]:3*layers[1]+3*layers[0]+3*layers[2]])
        self.layer4 = self._make_layer(block, inplanes*8, layers[3], stride=stride[2], dilation=dilation_factor, cfg = cfg[3*layers[1]+3*layers[0]+3*layers[2]:3*layers[1]+3*layers[0]+3*layers[2]+3*layers[3]])

        out_feature_strides = {'conv1': 4, 'layer1': 4, 'layer2': 4*stride[0], 'layer3': 4*stride[0]*stride[1],
                               'layer4': 4*stride[0]*stride[1]*stride[2]}

        # TODO better way?
        if isinstance(self.layer1[0], BasicBlock):
            out_feature_channels = {'conv1': inplanes, 'layer1': inplanes, 'layer2': inplanes*2, 'layer3': inplanes*4,
                               'layer4': inplanes*8}
        elif isinstance(self.layer1[0], Bottleneck):
            base_num_channels = 4 * inplanes
            out_feature_channels = {'conv1': inplanes, 'layer1': base_num_channels, 'layer2': base_num_channels * 2,
                                    'layer3': base_num_channels * 4, 'layer4': base_num_channels * 8}
        else:
            raise Exception('block not supported')

        self._out_feature_strides = out_feature_strides
        self._out_feature_channels = out_feature_channels

        # self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(cfg[-1], num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def out_feature_strides(self, layer=None):
        if layer is None:
            return self._out_feature_strides
        else:
            return self._out_feature_strides[layer]

    def out_feature_channels(self, layer=None):
        if layer is None:
            return self._out_feature_channels
        else:
            return self._out_feature_channels[layer]

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, cfg=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
                channel_selection(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, cfg[0:3], stride, downsample, dilation=dilation))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cfg[3*i:3*(i+1)]))

        return nn.Sequential(*layers)

    def _add_output_and_check(self, name, x, outputs, output_layers):
        if name in output_layers:
            outputs[name] = x
        return len(output_layers) == len(outputs)

    def forward(self, x, output_layers=None):
        """ Forward pass with input x. The output_layers specify the feature blocks which must be returned """
        outputs = OrderedDict()

        if output_layers is None:
            output_layers = self.output_layers

        x = self.conv1(x)
        x = self.bn1(x) ####### select daal dena
        x = self.relu(x)

        if self._add_output_and_check('conv1', x, outputs, output_layers):
            return outputs

        x = self.maxpool(x)

        x = self.layer1(x)

        if self._add_output_and_check('layer1', x, outputs, output_layers):
            return outputs

        x = self.layer2(x)

        if self._add_output_and_check('layer2', x, outputs, output_layers):
            return outputs

        x = self.layer3(x)

        if self._add_output_and_check('layer3', x, outputs, output_layers):
            return outputs

        x = self.layer4(x)

        if self._add_output_and_check('layer4', x, outputs, output_layers):
            return outputs

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        if self._add_output_and_check('fc', x, outputs, output_layers):
            return outputs

        if len(output_layers) == 1 and output_layers[0] == 'default':
            return x

        raise ValueError('output_layer is wrong.')


def resnet_baby(output_layers=None, pretrained=False, inplanes=16, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, inplanes=inplanes, **kwargs)

    if pretrained:
        raise NotImplementedError
    return model


def resnet18(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, **kwargs)

    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet50(output_layers=None, pretrained=False,cfg = None,**kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))
    model = ResNet(Bottleneck, [3, 4, 6, 3],output_layers,cfg=cfg,**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50'], progress = False), strict = False )
    return model


In [15]:
cfg.pop(1)

'M'

In [16]:
cfg

[62,
 47,
 53,
 133,
 200,
 60,
 63,
 63,
 63,
 67,
 127,
 128,
 221,
 224,
 109,
 144,
 116,
 118,
 168,
 123,
 127,
 147,
 256,
 255,
 569,
 299,
 247,
 247,
 216,
 247,
 181,
 233,
 242,
 208,
 238,
 242,
 254,
 251,
 254,
 329,
 512,
 512,
 2048,
 2048,
 512,
 2048,
 512,
 512,
 2048]

In [17]:
model_5 = resnet50(cfg=cfg)

In [18]:
import os
newmodel = resnet50(cfg=cfg)
old_modules = list(model.modules())
new_modules = list(newmodel.modules())
layer_id_in_cfg = 0
start_mask = torch.ones(3)
end_mask = cfg_mask[layer_id_in_cfg]
conv_count = 0

for layer_id in range(len(old_modules)):
    m0 = old_modules[layer_id]
    m1 = new_modules[layer_id]
    if isinstance(m0, nn.BatchNorm2d):
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.size == 1:
            idx1 = np.resize(idx1,(1,))

        if isinstance(old_modules[layer_id + 1], channel_selection):
            # If the next layer is the channel selection layer, then the current batchnorm 2d layer won't be pruned.
            m1.weight.data = m0.weight.data.clone()
            m1.bias.data = m0.bias.data.clone()
            m1.running_mean = m0.running_mean.clone()
            m1.running_var = m0.running_var.clone()

            # We need to set the channel selection layer.
            m2 = new_modules[layer_id + 1]
            m2.indexes.data.zero_()
            m2.indexes.data[idx1.tolist()] = 1.0

            layer_id_in_cfg += 1
            start_mask = end_mask.clone()
            if layer_id_in_cfg < len(cfg_mask):
                end_mask = cfg_mask[layer_id_in_cfg]
        else:
            m1.weight.data = m0.weight.data[idx1.tolist()].clone()
            m1.bias.data = m0.bias.data[idx1.tolist()].clone()
            m1.running_mean = m0.running_mean[idx1.tolist()].clone()
            m1.running_var = m0.running_var[idx1.tolist()].clone()
            layer_id_in_cfg += 1
            start_mask = end_mask.clone()
            if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
                end_mask = cfg_mask[layer_id_in_cfg]

    
    elif isinstance(m0, nn.Conv2d):
        if conv_count == 0:
            m1.weight.data = m0.weight.data.clone()
            conv_count += 1
            print("BAHAR ka conv")
            continue
        if isinstance(old_modules[layer_id+2], nn.Conv2d):
            # Residual first case
            print("CASE 1 ")
            conv_count += 1
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
            print('In shape: {:d}, Out shape {:d}**.'.format(idx0.size, idx1.size))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            if idx1.size == 1:
                idx1 = np.resize(idx1, (1,))
            w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
            w1 = w1[idx1.tolist(), :, :, :].clone()
            m1.weight.data = w1.clone()
#             if conv_count % 3 != 1:           
            continue
        if isinstance(old_modules[layer_id+3], nn.ReLU) and isinstance(old_modules[layer_id+2], channel_selection):
            # Residualsecond case
            print("CASE 2 ")
            conv_count += 1
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
            print('In shape: {:d}, Out shape {:d}.'.format(idx0.size, idx1.size))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            if idx1.size == 1:
                idx1 = np.resize(idx1, (1,))
            w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
            m1.weight.data = w1.clone()
            continue
        print("DOWN ka conv")
        m1.weight.data = m0.weight.data.clone()
 
    elif isinstance(m0, nn.Linear):
        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        if idx0.size == 1:
            idx0 = np.resize(idx0, (1,))

        m1.weight.data = m0.weight.data[:, idx0].clone()
        m1.bias.data = m0.bias.data.clone()

BAHAR ka conv
CASE 1 
In shape: 62, Out shape 47**.
CASE 1 
In shape: 47, Out shape 53**.
CASE 2 
In shape: 53, Out shape 133.
DOWN ka conv
CASE 1 
In shape: 200, Out shape 56**.
CASE 1 
In shape: 56, Out shape 60**.
CASE 2 
In shape: 60, Out shape 63.
CASE 1 
In shape: 63, Out shape 63**.
CASE 1 
In shape: 63, Out shape 63**.
CASE 2 
In shape: 63, Out shape 67.
CASE 1 
In shape: 67, Out shape 127**.
CASE 1 
In shape: 127, Out shape 128**.
CASE 2 
In shape: 128, Out shape 221.
DOWN ka conv
CASE 1 
In shape: 224, Out shape 43**.
CASE 1 
In shape: 43, Out shape 109**.
CASE 2 
In shape: 109, Out shape 144.
CASE 1 
In shape: 144, Out shape 116**.
CASE 1 
In shape: 116, Out shape 118**.
CASE 2 
In shape: 118, Out shape 168.
CASE 1 
In shape: 168, Out shape 123**.
CASE 1 
In shape: 123, Out shape 127**.
CASE 2 
In shape: 127, Out shape 147.
CASE 1 
In shape: 147, Out shape 256**.
CASE 1 
In shape: 256, Out shape 255**.
CASE 2 
In shape: 255, Out shape 569.
DOWN ka conv
CASE 1 
In shape: 299,

In [19]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()},'/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/dimp50_correct.pth.tar')

In [21]:
newmodel.load_state_dict(torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/dimp50_correct.pth.tar')['state_dict'])

<All keys matched successfully>

In [22]:
cfg1 = torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/dimp50_correct.pth.tar')['cfg']

In [23]:
newmodel = resnet50(cfg = cfg1)


In [25]:
new = resnet50(cfg = cfg1)
new.load_state_dict(torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/dimp50_correct.pth.tar')['state_dict'])

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 3, 7, 7]) from checkpoint, the shape in current model is torch.Size([62, 3, 7, 7]).
	size mismatch for layer1.1.conv1.weight: copying a param with shape torch.Size([56, 200, 1, 1]) from checkpoint, the shape in current model is torch.Size([200, 133, 1, 1]).
	size mismatch for layer1.1.bn1.weight: copying a param with shape torch.Size([56]) from checkpoint, the shape in current model is torch.Size([200]).
	size mismatch for layer1.1.bn1.bias: copying a param with shape torch.Size([56]) from checkpoint, the shape in current model is torch.Size([200]).
	size mismatch for layer1.1.bn1.running_mean: copying a param with shape torch.Size([56]) from checkpoint, the shape in current model is torch.Size([200]).
	size mismatch for layer1.1.bn1.running_var: copying a param with shape torch.Size([56]) from checkpoint, the shape in current model is torch.Size([200]).


In [ ]:
ckpt = torch.load('pruned.pth.tar')['state_dict']

In [ ]:
ckpt['conv1.weight'].shape

In [ ]:
conv
bn

layer1
    3conv 3bn
    X3
layer2
    3conv 3bn
    X4
layer3
    3conv 3bn
    X6
layer4
    3conv 3bn
    X3



In [ ]:
new

In [ ]:
newmodel